In [18]:
import numpy as np
import matplotlib.pyplot as plt
import math
from ipywidgets import interact, FloatSlider, IntSlider, Checkbox, interactive, Text, FloatText
from scipy.interpolate import splprep, splev
from datetime import datetime

def kresling_crease(
    dia, pattern_height, green_red_angle, floors, n,
    ext_lenght=2.0, seam_lenght=0.5, seam_lenght_left=0.5,
    x_offset=0, y_offset=0,
    export_pdf=False, file_name="kresling_pattern_A4.pdf"
):
    """
    Draws a Kresling origami crease pattern and optionally exports to A4 PDF with 1 unit = 1 cm.
    x_offset and y_offset shift the pattern in cm.
    The green-red angle is the angle between the diagonal (green) and vertical (red) creases.
    The construction is from green to red: green-red angle and geometry are fixed, red zigzag is derived.
    Also draws orange lines extending from the four corners, with adjustable length.
    """
    pattern_width = 2 * (dia / 2) * np.pi
    b = pattern_width / n  # Horizontal unit (blue)
    floor_height = pattern_height / floors  # Vertical unit

    # Calculate dx so that the angle between the green (diagonal) and red (vertical) is green_red_angle
    phi = np.deg2rad(green_red_angle)
    # tan(phi) = (b + 2*dx) / floor_height  =>  dx = (floor_height * tan(phi) - b) / 2
    dx = (floor_height * np.tan(phi) - b) / 2
    green_dx = b + 2*dx
    green_length = math.sqrt(green_dx**2 + floor_height**2)

    # The red zigzag connects (x, y) to (x + dx, y + floor_height)
    red_dx = b + 2*dx
    red_length = math.sqrt(red_dx**2 + floor_height**2)

    # Calculate blue-red angle (angle between blue and red, for info)
    # Blue is horizontal, red is the vertical zigzag (dx offset per floor)
    # The red zigzag goes from (x, y) to (x + dx, y + floor_height)
    # So the angle between blue (horizontal) and red (zigzag) is:
    # theta = atan2(dx, floor_height)
    # But we want 90° when perpendicular, so:
    blue_red_angle = 90 - abs(math.degrees(math.atan2(dx, floor_height)))

    # Calculate blue-green angle (angle between blue and green)
    # Green goes from (x, y) to (x + green_dx, y + floor_height)
    blue_green_angle = abs(math.degrees(math.atan2(green_dx, floor_height)))

    # Output floor height, one side length, red line length, and green-red angle
    print(f"Floor height: {floor_height:.4f} cm")
    print(f"One side length (blue): {b:.4f} cm")
    print(f"Red line length: {red_length:.4f} cm")
    print(f"Blue-Green Angle: {blue_green_angle:.2f}°")
    print(f"Green-Red angle (input): {green_red_angle:.2f}°")
    print(f"Blue-Red angle (computed): {abs(blue_red_angle):.2f}°")
    print(f"Red / Radius: {red_length / (dia/2)}")

    # A4 size in cm: 21 x 29.7
    fig_width_cm = 21
    fig_height_cm = 29.7

    # Set figure size in cm (matplotlib uses inches, so convert)
    fig, ax = plt.subplots(figsize=(fig_width_cm/2.54, fig_height_cm/2.54), dpi=72)  # dpi=72 for true 1:1 cm on PDF

    # Calculate scaling factor to match actual printed floor height
    intended_floor_height = floor_height
    measured_floor_height = 1.9  # Adjust this value to your measured result
    scale_factor = intended_floor_height / measured_floor_height

    # Store vertices
    vertices = []
    for floor in range(floors + 1):
        row_vertices = []
        for col in range(n + 1):
            x = col * b
            y = floor * floor_height
            # Apply horizontal shift for zigzag pattern
            if floor % 2 == 1:
                x += dx
            else:
                x -= dx
            # Apply user offset
            x += x_offset
            y += y_offset
            # Apply scaling
            x *= scale_factor
            y *= scale_factor
            row_vertices.append((x, y))
        vertices.append(row_vertices)

    # Draw vertical red zigzag creases (skip left/right edges)
    for col in range(1, n):  # skip col=0 and col=n
        x_vals = [vertices[floor][col][0] for floor in range(floors + 1)]
        y_vals = [vertices[floor][col][1] for floor in range(floors + 1)]
        ax.plot(x_vals, y_vals, 'r')

    # # Draw left and right red edge curves
    # for edge_col in [0, n]:
    #     x_edge = [vertices[floor][edge_col][0] for floor in range(floors + 1)]
    #     y_edge = [vertices[floor][edge_col][1] for floor in range(floors + 1)]
    #     tck, u = splprep([x_edge, y_edge], s=0)
    #     unew = np.linspace(0, 1, 200)
    #     out = splev(unew, tck)
    #     ax.plot(out[0], out[1], 'r', lw=2)  # Thicker red curve

    # Draw red dot line curve only for first and final floor (top and bottom)
    # Top edge (first floor)
    # x_top = [vertices[0][col][0] for col in range(n + 1)]
    # y_top = [vertices[0][col][1] for col in range(n + 1)]
    # ax.plot(x_top, y_top, color='r', linestyle=':', linewidth=2)

    # # Bottom edge (final floor)
    # x_bot = [vertices[-1][col][0] for col in range(n + 1)]
    # y_bot = [vertices[-1][col][1] for col in range(n + 1)]
    # ax.plot(x_bot, y_bot, color='r', linestyle=':', linewidth=2)

    # Draw line along right red edge curve
    x_right = [vertices[floor][n][0] for floor in range(floors + 1)]
    y_right = [vertices[floor][n][1] for floor in range(floors + 1)]
    ax.plot(x_right, y_right, color='r', linestyle=(0, (1, 1)), linewidth=2)

    # Draw line along left red edge curve
    x_left = [vertices[floor][0][0] for floor in range(floors + 1)]
    y_left = [vertices[floor][0][1] for floor in range(floors + 1)]
    ax.plot(x_left, y_left, color='r', linestyle=(0, (1, 1)), linewidth=2)

    # Draw horizontal blue creases
    for floor in range(floors + 1):
        x_vals = [vertices[floor][col][0] for col in range(n + 1)]
        y_vals = [vertices[floor][col][1] for col in range(n + 1)]
        ax.plot(x_vals, y_vals, 'b')

    # Draw diagonal green creases in a zigzag pattern (reverse direction):
    for floor in range(floors):
        for col in range(n):
            v1 = vertices[floor][col]
            v2 = vertices[floor + 1][col + 1]
            v3 = vertices[floor][col + 1]
            v4 = vertices[floor + 1][col]
            if floor % 2 == 0:
                # Even row: \
                ax.plot([v3[0], v4[0]], [v3[1], v4[1]], 'g--', alpha=0.8)
            else:
                # Odd row: /
                ax.plot([v1[0], v2[0]], [v1[1], v2[1]], 'g--', alpha=0.8)

    # Draw purple vertical lines extending out of each corner (not through the crease)
    # Top-left
    x0, y0 = vertices[0][0]
    # Top-right
    x1, y1 = vertices[0][-1]
    # Bottom-left
    x2, y2 = vertices[-1][0]
    # Bottom-right
    x3, y3 = vertices[-1][-1]

    # Top verticals
    ax.plot([x0, x0], [y0 - ext_lenght, y0], color='purple', lw=2)
    ax.plot([x1, x1], [y1 - ext_lenght, y1], color='purple', lw=2)
    # Bottom verticals
    ax.plot([x2, x2], [y2, y2 + ext_lenght], color='purple', lw=2)
    ax.plot([x3, x3], [y3, y3 + ext_lenght], color='purple', lw=2)

    # Seam Extend ('slateblue' dot)
    # Right
    ax.plot([x1, x1 + seam_lenght], [y0, y0], color='slateblue', linestyle=(0, (1, 1)), lw=2)
    ax.plot([x1, x1 + seam_lenght], [y1 - ext_lenght, y1 - ext_lenght], color='slateblue', linestyle=(0, (1, 1)), lw=2)
    ax.plot([x1, x1 + seam_lenght], [y2, y2], color='slateblue', linestyle=(0, (1, 1)), lw=2)
    ax.plot([x1, x1 + seam_lenght], [y3 + ext_lenght, y3 + ext_lenght], color='slateblue', linestyle=(0, (1, 1)), lw=2)
    # Vertical right
    ax.plot([x1 + seam_lenght, x1 + seam_lenght], [y0 - ext_lenght, y3 + ext_lenght], color='slateblue', linestyle=(0, (1, 1)), lw=2)

    # Left
    ax.plot([x0 - seam_lenght_left, x0], [y0, y0], color='slateblue', linestyle=(0, (1, 1)), lw=2)
    ax.plot([x0 - seam_lenght_left, x0], [y0 - ext_lenght, y0 - ext_lenght], color='slateblue', linestyle=(0, (1, 1)), lw=2)
    ax.plot([x0 - seam_lenght_left, x0], [y2, y2], color='slateblue', linestyle=(0, (1, 1)), lw=2)
    ax.plot([x0 - seam_lenght_left, x0], [y3 + ext_lenght, y3 + ext_lenght], color='slateblue', linestyle=(0, (1, 1)), lw=2)
    # Vertical left
    ax.plot([x0 - seam_lenght_left, x0 - seam_lenght_left], [y0 - ext_lenght, y3 + ext_lenght], color='slateblue', linestyle=(0, (1, 1)), lw=2)

    # Connect the ends of the left and right purple lines (top and bottom)
    # Top: from (x0, y0 - ext_lenght) to (x1, y1 - ext_lenght)
    ax.plot([x0, x1], [y0 - ext_lenght, y1 - ext_lenght], color='purple', lw=2)
    # Bottom: from (x2, y2 + ext_lenght) to (x3, y3 + ext_lenght)
    ax.plot([x2, x3], [y2 + ext_lenght, y3 + ext_lenght], color='purple', lw=2)

    # Move information box inside the purple rectangle (top left, just inside the purple box)
    info_x = x0 + 0.3
    info_y = y0 - ext_lenght + 1.8

    export_date = datetime.now().strftime('%Y-%m-%d %H:%M') if export_pdf else ''
    settings_text = (
        f"Floor height: {floor_height:.2f} cm\n"
        f"One side: {b:.2f} cm\n"
        f"Red line: {red_length:.2f} cm\n"
        f"Blue-Green Angle: {blue_green_angle:.2f}°\n"
        f"Green-Red angle: {green_red_angle:.2f}°\n"
        f"Blue-Red Angle: {abs(blue_red_angle):.2f}°\n"
    )
    ax.text(
        info_x, info_y, settings_text,
        fontsize=6, ha='left', va='top', family='monospace',
        bbox=dict(alpha=0)  # No background
    )

    info2_x = x1 - 0.3
    info2_y = y0 - ext_lenght + 1.8
    settings_text2 = (
        f"Diameter: {dia:.2f} cm\n"
        f"Height: {pattern_height} cm\n"
        f"Total length {pattern_width:.2f} cm\n"
        f"Floors: {floors}\n"
        f"Sides: {n}\n"
        f"File: {file_name}"
        + (f"\nExported: {export_date}" if export_pdf else "")
    )
    ax.text(
        info2_x, info2_y, settings_text2,
        fontsize=6, ha='right', va='top', family='monospace',
        bbox=dict(alpha=0)
    )

    # Set axis limits to fit A4 and center the pattern
    ax.set_xlim(0, fig_width_cm)
    ax.set_ylim(0, fig_height_cm)
    ax.set_aspect('equal', adjustable='box')  # Ensure 1:1 scaling in cm
    # Hide axis ticks, labels, and spines for clean output
    ax.set_xticks([])
    ax.set_yticks([])
    # Remove axis labels and axis lines
    ax.axis('off')

    plt.tight_layout()

    if export_pdf:
        plt.savefig(file_name, format="pdf")  # Remove bbox_inches='tight' for true scale
        print(f"Exported to {file_name}")
        plt.close(fig)
    else:
        plt.show()

# Use interactive instead of interact for immediate recalculation on slider change
ui = interactive(
    kresling_crease,
    dia=FloatSlider(value=3, min=1, max=10, step=0.5, description='Diameter (cm)'),
    pattern_height=FloatSlider(value=20, min=5, max=40, step=1, description='Height (cm)'),
    green_red_angle=FloatSlider(value=6.7, min=1, max=70, step=0.1, description='Green-Red Angle (°)'),
    floors=IntSlider(value=10, min=1, max=20, step=1, description='Floors'),
    n=IntSlider(value=6, min=1, max=20, step=1, description='Sides'),
    ext_lenght=FloatSlider(value=2.0, min=0.1, max=10, step=0.1, description='Ends Ext (cm)'),
    seam_lenght=FloatSlider(value=0, min=0, max=10, step=0.1, description='Seam Ext Right (cm)'),
    seam_lenght_left=FloatSlider(value=2, min=0, max=10, step=0.1, description='Seam Ext Left (cm)'),
    x_offset=FloatSlider(value=3.5, min=-10, max=10, step=0.1, description='X Offset (cm)'),
    y_offset=FloatSlider(value=3, min=-10, max=10, step=0.1, description='Y Offset (cm)'),
    export_pdf=Checkbox(value=False, description='Export to PDF'),
    file_name=Text(value='kresling_pattern_A4.pdf', description='File Name')
)
display(ui)

interactive(children=(FloatSlider(value=3.0, description='Diameter (cm)', max=10.0, min=1.0, step=0.5), FloatS…